In [1]:
#referencia https://medium.com/towards-artificial-intelligence/multi-class-text-classification-using-pyspark-mllib-doc2vec-dbfcee5b39f2
import findspark
findspark.init() 
from pyspark.sql import SparkSession
import os


MAX_MEMORY = "8g"
spark = SparkSession.builder \
                    .master("local") \
                    .appName('va_repositorio')\
                    .config("spark.executor.memory", MAX_MEMORY) \
                    .config("spark.driver.memory", MAX_MEMORY) \
                    .config("spark.jars", "E:\\va\\driverdb\\postgresql-42.2.5.jre6") \
                    .getOrCreate()

In [2]:
tablas=["archivos.ftp_sbpro_gen_20121","archivos.ftp_sbpro_gen_20122","archivos.ftp_sbpro_gen_20131",
        "archivos.ftp_sbpro_gen_20132","archivos.ftp_sbpro_gen_20141","archivos.ftp_sbpro_gen_20142",
        "archivos.ftp_sbpro_gen_20151","archivos.ftp_sbpro_gen_20152","archivos.ftp_sbpro_gen_2016"
       ,"archivos.ftp_sbpro_gen_2017","archivos.ftp_sbpro_gen_2018"]
datos=[]
for t in tablas:
    df  = spark.read \
        .format("jdbc") \
        .option("url", "jdbc:postgresql://localhost:5432/va") \
        .option("dbtable", t) \
        .option("user", "postgres") \
        .option("password", "david") \
        .option("driver", "org.postgresql.Driver") \
        .load()
    df=df.where("inst_nombre_institucion = 'UNIVERSIDAD DE NARIÑO-PASTO' or inst_nombre_institucion='UNIVERSIDAD DE NARIÑO'")
    datos.append(df)
    print(df.count())

381
737
417
1146
169
1125
79
1107
1189
1252
997


In [3]:
variables=["periodo","estu_mcpio_reside","estu_genero","estu_estadocivil","estu_etnia","estu_limita_motriz","estu_limita_invidente",
          "estu_limita_sordoconinterprete","estu_limita_sordosininterprete","estu_limita_sordoceguera",
          "estu_porcentajecreditosaprob","estu_valormatriculauniversidad","estu_pagomatriculabeca",
          "estu_pagomatriculacredito","estu_pagomatriculapadres","estu_pagomatriculapropio","estu_tomo_cursopreparacion",
          "estu_cursodocentesies","estu_cursoiesapoyoexterno","estu_cursoiesexterna","estu_simulacrotipoicfes",
          "estu_actividadrefuerzoareas","estu_actividadrefuerzogeneric","estu_semestrecursa","fami_hogaractual",
          "fami_cabezafamilia","fami_numpersonasacargo","fami_educacionpadre","fami_educacionmadre","fami_ocupacionpadre",
          "fami_ocupacionmadre","fami_estratovivienda","fami_nivel_sisben","fami_personashogar","fami_cuartoshogar",
          "fami_pisos_hogar","fami_tieneinternet","fami_tiene_serviciotv","fami_tienecomputador","fami_tienelavadora",
          "fami_tiene_microondas","fami_tiene_horno","fami_tieneautomovil","fami_tiene_dvd","fami_tiene_nevera",
          "fami_tiene_celular","fami_telefono","fami_tienehornomicroogas","fami_tienemotocicleta","fami_numlibros",
          "estu_dedicacionlecturadiaria","estu_dedicacioninternet","fami_tieneconsolavideojuegos","fami_cuantoscompartebano",
          "fami_ingreso_fmiliar_mensual","estu_trabaja_actualmente","estu_horassemanatrabaja","estu_tiporemuneracion",
          "estu_prgm_academico","gruporeferencia","inst_origen","estu_privado_libertad","estu_nse_individual","estu_nse_ies"]

In [4]:
#metodo 2

def buscar_variable(variables,nombre):
    for v in variables:
        if v==nombre:
            return True
    return False

sql=[]
indice=0
for d in datos:
    columnas=""
    for v in variables:
        if(buscar_variable(d.columns,v)):
            columnas=columnas+v+","
        else:
            columnas=columnas+"'NO_APLICA'"+v+","
    columnas=columnas+"estu_consecutivo"   
    d.createOrReplaceTempView("icfes"+str(indice))
    s="select "+columnas+" from " +"icfes"+str(indice)
    sql.append(s)
    print(s)
    indice=indice+1
        

select periodo,estu_mcpio_reside,estu_genero,estu_estadocivil,'NO_APLICA'estu_etnia,'NO_APLICA'estu_limita_motriz,'NO_APLICA'estu_limita_invidente,'NO_APLICA'estu_limita_sordoconinterprete,'NO_APLICA'estu_limita_sordosininterprete,'NO_APLICA'estu_limita_sordoceguera,estu_porcentajecreditosaprob,estu_valormatriculauniversidad,estu_pagomatriculabeca,estu_pagomatriculacredito,estu_pagomatriculapadres,estu_pagomatriculapropio,'NO_APLICA'estu_tomo_cursopreparacion,'NO_APLICA'estu_cursodocentesies,'NO_APLICA'estu_cursoiesapoyoexterno,'NO_APLICA'estu_cursoiesexterna,'NO_APLICA'estu_simulacrotipoicfes,'NO_APLICA'estu_actividadrefuerzoareas,'NO_APLICA'estu_actividadrefuerzogeneric,estu_semestrecursa,fami_hogaractual,fami_cabezafamilia,fami_numpersonasacargo,fami_educacionpadre,fami_educacionmadre,fami_ocupacionpadre,fami_ocupacionmadre,fami_estratovivienda,fami_nivel_sisben,fami_personashogar,fami_cuartoshogar,fami_pisos_hogar,fami_tieneinternet,fami_tiene_serviciotv,fami_tienecomputador,fami_t

In [6]:
sql_final=""
cont=0
for s in sql:
    if(cont==len(datos)-1):
        sql_final=sql_final+s
    else:
         sql_final=sql_final+" "+s+" union "
    print(cont)
    cont=cont+1
sql_final

0
1
2
3
4
5
6
7
8
9
10


" select periodo,estu_mcpio_reside,estu_genero,estu_estadocivil,'NO_APLICA'estu_etnia,'NO_APLICA'estu_limita_motriz,'NO_APLICA'estu_limita_invidente,'NO_APLICA'estu_limita_sordoconinterprete,'NO_APLICA'estu_limita_sordosininterprete,'NO_APLICA'estu_limita_sordoceguera,estu_porcentajecreditosaprob,estu_valormatriculauniversidad,estu_pagomatriculabeca,estu_pagomatriculacredito,estu_pagomatriculapadres,estu_pagomatriculapropio,'NO_APLICA'estu_tomo_cursopreparacion,'NO_APLICA'estu_cursodocentesies,'NO_APLICA'estu_cursoiesapoyoexterno,'NO_APLICA'estu_cursoiesexterna,'NO_APLICA'estu_simulacrotipoicfes,'NO_APLICA'estu_actividadrefuerzoareas,'NO_APLICA'estu_actividadrefuerzogeneric,estu_semestrecursa,fami_hogaractual,fami_cabezafamilia,fami_numpersonasacargo,fami_educacionpadre,fami_educacionmadre,fami_ocupacionpadre,fami_ocupacionmadre,fami_estratovivienda,fami_nivel_sisben,fami_personashogar,fami_cuartoshogar,fami_pisos_hogar,fami_tieneinternet,fami_tiene_serviciotv,fami_tienecomputador,fami

In [7]:
df_repositorio=spark.sql(sql_final)

In [10]:
#for v in variables:
   # df_repositorio.groupby(v).count().show(5)
df_repositorio.count()

8599

In [9]:
df_repositorio.write.jdbc(url="jdbc:postgresql://localhost:5432/va"
                  "?user=postgres&password=david",
              table="proyecto.va_repositorio_inicial",
              mode="overwrite",
              properties={"driver": 'org.postgresql.Driver'})